In [1]:
!pip install cplex
!pip install docplex
from docplex.mp.model import Model

     |████████████████████████████████| 39.6 MB 44.3 MB/s eta 0:00:01    |█████████▎                      | 11.5 MB 4.0 MB/s eta 0:00:08     |█████████████▋                  | 16.8 MB 4.0 MB/s eta 0:00:06
     |████████████████████████████████| 610 kB 3.7 MB/s eta 0:00:01
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=bb841f26cb8fc3a066d0e86142b31d01fd9bffc9bc3262acf953a0cb540507b9
  Stored in directory: /Users/joaparodi/Library/Caches/pip/wheels/5e/4c/27/b3b3229ab9a25a089d5c01551d7c2e7efa41462635066784a0
Successfully built docplex


In [2]:
# CLASES
class Clothing:
    def __init__(self, number):
        self.number = number
        self.incompatibles = {}
        self.washing_time = 0

    def add_incompatible(self, clothing_to_add: "Clothing"):
        self.incompatibles[clothing_to_add.number] = clothing_to_add

    def __str__(self):
        return str(self.number)

class Washing:
    def __init__(self, clothing: Clothing):
        self.clothings = [clothing]
        self.washing_time = clothing.washing_time

    def add_clothing(self, new_clothing: Clothing):
        if any(new_clothing in x.incompatibles for x in self.clothings):
            return False
        self.clothings.append(new_clothing)
        if new_clothing.washing_time > self.washing_time:
            self.washing_time = new_clothing.washing_time
        return True

    def can_add_clothing(self, new_clothing: Clothing):
        if any(new_clothing.number in clothing.incompatibles for clothing in self.clothings):
            return False
        return True

    def __contains__(self, key):
        return key in self.clothings

clothings_amount = 0

file = open("data.txt", "r+")
clothings_graph = {}
for current_line in file:
    if current_line[0] == 'p':
        clothings_amount = int(current_line.split(' ')[2])
        for i in range(clothings_amount):
            clothings_graph[i + 1] = Clothing(i + 1)
    elif current_line[0] == 'n':
        clothings_graph[int(current_line.split(' ')[1])].washing_time = int(current_line.split(' ')[2])
    elif current_line[0] == 'e':
        clothings_graph[int(current_line.split(' ')[1])].add_incompatible(clothings_graph[int(current_line.split(' ')[2])])
        clothings_graph[int(current_line.split(' ')[2])].add_incompatible(clothings_graph[int(current_line.split(' ')[1])])

In [3]:
T = {} # tiempo de cada lavado
I = {} # incompatibilidades entre prendas

for i in clothings_graph:
  T[i] = clothings_graph[i].washing_time

for i in clothings_graph:
  for j in clothings_graph:
    if i == j:
      I[(i,j)] = 0
    else:
      I[(i,j)] = 1
    
for i in clothings_graph:
  for j in clothings_graph[i].incompatibles:
    I[(i,j)] = 0
    I[(j,i)] = 0

mdl = Model('modelo')
# armo la matriz binaria
x = mdl.binary_var_matrix(range(1, clothings_amount + 1), range(1, clothings_amount + 1), name='x')
t = mdl.integer_var_dict(range(1, clothings_amount + 1), name='t')

# para que cada prenda este en un solo lavado
mdl.add_constraints((sum(x[(i,j)] for i in clothings_graph)) == 1 for j in clothings_graph)
# para respetar incompatibilidades entre prendas
for j in clothings_graph:
  mdl.add_constraint(t[j] == mdl.max(x[(i in range(1, clothings_amount + 1),j)]*T[j]))

for i in clothings_graph:
  for j in clothings_graph:
    mdl.add_constraint(x[(i,j)] <= I[(i,j)])

obj = sum(t[i] for i in t)
mdl.set_objective('min',obj)

mdl.print_solution()


In [4]:
solucion = mdl.solve(log_output=True)

CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 19182 vars, 19320 consts, CPLEX code=1016